### =============================================
### NOTEBOOK 01 - PRÉPARATION DES DONNÉES
### =============================================

### 📊 Préparation des Données pour Drone Agri AI

### Ce notebook couvre:
### 1. Téléchargement des datasets
### 2. Exploration et visualisation
### 3. Nettoyage et prétraitement
### 4. Augmentation des données
### 5. Division train/val/test

In [ ]:
# =============================================================================
# CONFIGURATION DE L'ENVIRONNEMENT (Compatible Local + Colab)
# =============================================================================

import os
import sys
from pathlib import Path

# === DÉTECTION DE L'ENVIRONNEMENT ===
IN_COLAB = 'google.colab' in sys.modules

print(f"🖥️ Environnement: {'Google Colab' if IN_COLAB else 'Local'}")

# === CONFIGURATION DES CHEMINS ===
if IN_COLAB:
    # --- COLAB ---
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Chemin personnalisable par l'utilisateur
    DRIVE_BASE = '/content/drive/MyDrive/Stage-X4/Drone-Agricole'
    
    PROJECT_DIR = Path('/content/drone-agri-ai')
    DATA_DIR = Path('/content/data')
    SAVE_DIR = Path(DRIVE_BASE) / 'drone-agri-ai'
else:
    # --- LOCAL ---
    # Déterminer le dossier du projet
    if Path.cwd().name == 'notebooks':
        PROJECT_DIR = Path.cwd().parent
    else:
        PROJECT_DIR = Path.cwd()
    
    DATA_DIR = PROJECT_DIR / 'data'
    SAVE_DIR = PROJECT_DIR

# Dossiers communs
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
MODELS_DIR = SAVE_DIR / 'models'

# Créer les dossiers
for d in [DATA_DIR, RAW_DIR, PROCESSED_DIR, MODELS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print(f"📂 Projet: {PROJECT_DIR}")
print(f"📂 Données: {DATA_DIR}")
print(f"📂 Sauvegarde: {SAVE_DIR}")
print(f"📂 Modèles: {MODELS_DIR}")

In [ ]:
# =============================================================================
# TÉLÉCHARGEMENT DES DATASETS (Kaggle ou lien Drive)
# =============================================================================

import os
from pathlib import Path

# === CONFIGURATION ===
# Option 1: Téléchargement Kaggle (nécessite kaggle.json)
# Option 2: Lien Google Drive (pour utilisateur local sans Kaggle)

USE_KAGGLE = True  # ← Mettre False si tu veux utiliser un lien Drive

# Lien Google Drive vers un zip des datasets (optionnel)
DRIVE_DATASET_URL = ""  # ← Mettre l'ID du fichier si USE_KAGGLE = False
# Exemple: "1ABC123xyz" pour https://drive.google.com/file/d/1ABC123xyz/view

# === LISTE DES DATASETS À TÉLÉCHARGER ===
DATASETS = [
    ("abdallahalidev/plantvillage-dataset", "plantvillage"),
    ("vipoooool/new-plant-diseases-dataset", "new-plant-diseases"),
    ("vbookshelf/v2-plant-seedlings-dataset", "plant-seedlings"),
]

def download_with_kaggle():
    """Télécharge les datasets via Kaggle API"""
    import subprocess
    
    # Configurer Kaggle
    kaggle_dir = Path.home() / '.kaggle'
    kaggle_dir.mkdir(exist_ok=True)
    
    # Chercher kaggle.json
    possible_paths = [
        kaggle_dir / 'kaggle.json',
        Path('/content/drive/MyDrive/kaggle.json') if IN_COLAB else None,
        Path('/content/drive/MyDrive/Stage-X4/Drone-Agricole/kaggle.json') if IN_COLAB else None,
        PROJECT_DIR / 'kaggle.json',
        Path.home() / 'kaggle.json',
    ]
    
    kaggle_json = None
    for p in possible_paths:
        if p and p.exists():
            kaggle_json = p
            break
    
    if kaggle_json is None:
        print("❌ kaggle.json non trouvé!")
        print("📋 Instructions:")
        print("   1. Va sur https://www.kaggle.com → Settings → API → Create New Token")
        print("   2. Place kaggle.json dans:", kaggle_dir)
        return False
    
    # Copier vers ~/.kaggle si nécessaire
    if kaggle_json != kaggle_dir / 'kaggle.json':
        import shutil
        shutil.copy(kaggle_json, kaggle_dir / 'kaggle.json')
    os.chmod(kaggle_dir / 'kaggle.json', 0o600)
    
    print("✅ Kaggle configuré!")
    
    # Télécharger chaque dataset
    for dataset_name, folder_name in DATASETS:
        dest = RAW_DIR / folder_name
        dest.mkdir(exist_ok=True)
        
        # Vérifier si déjà téléchargé
        if any(dest.iterdir()):
            print(f"⏭️ {folder_name} déjà présent, skip...")
            continue
        
        print(f"📥 Téléchargement: {dataset_name}...")
        try:
            subprocess.run([
                'kaggle', 'datasets', 'download', '-d', dataset_name,
                '-p', str(dest), '--unzip', '-q'
            ], check=True)
            print(f"   ✅ {folder_name} téléchargé!")
        except Exception as e:
            print(f"   ⚠️ Erreur: {e}")
    
    return True

def download_from_drive(file_id):
    """Télécharge depuis Google Drive (pour utilisateurs sans Kaggle)"""
    import urllib.request
    import zipfile
    
    url = f"https://drive.google.com/uc?export=download&id={file_id}"
    zip_path = DATA_DIR / 'datasets.zip'
    
    print(f"📥 Téléchargement depuis Google Drive...")
    urllib.request.urlretrieve(url, zip_path)
    
    print("📦 Extraction...")
    with zipfile.ZipFile(zip_path, 'r') as z:
        z.extractall(RAW_DIR)
    
    zip_path.unlink()
    print("✅ Datasets extraits!")
    return True

# === EXÉCUTION ===
if USE_KAGGLE:
    success = download_with_kaggle()
else:
    if DRIVE_DATASET_URL:
        success = download_from_drive(DRIVE_DATASET_URL)
    else:
        print("⚠️ Aucune source de données configurée!")
        print("   Mets USE_KAGGLE = True ou fournis DRIVE_DATASET_URL")
        success = False

if success:
    print("\n📁 Contenu de RAW_DIR:")
    for item in RAW_DIR.iterdir():
        if item.is_dir():
            count = sum(1 for _ in item.rglob('*') if _.is_file())
            print(f"   📂 {item.name}: {count} fichiers")

In [ ]:
# === TROUVER LE DOSSIER PRINCIPAL DES IMAGES ===

def find_data_folder(base_dir):
    """Trouve automatiquement le dossier contenant les classes d'images"""
    base_dir = Path(base_dir)
    
    # Chemins possibles selon la structure des datasets
    candidates = [
        base_dir / 'plantvillage' / 'plantvillage dataset' / 'color',
        base_dir / 'plantvillage' / 'PlantVillage',
        base_dir / 'plantvillage' / 'color',
        base_dir / 'new-plant-diseases' / 'New Plant Diseases Dataset(Augmented)' / 'train',
        base_dir / 'new-plant-diseases' / 'train',
        base_dir / 'color',
        base_dir / 'train',
    ]
    
    for path in candidates:
        if path.exists():
            subdirs = [d for d in path.iterdir() if d.is_dir()]
            if len(subdirs) > 5:  # Au moins quelques classes
                return path
    
    # Recherche automatique récursive
    for root, dirs, files in os.walk(base_dir):
        if len(dirs) > 15:  # Beaucoup de sous-dossiers = classes
            sample = Path(root) / dirs[0]
            images = list(sample.glob('*.jpg')) + list(sample.glob('*.JPG')) + list(sample.glob('*.png'))
            if len(images) > 5:
                return Path(root)
    
    return None

# Trouver le dossier
data_path = find_data_folder(RAW_DIR)

if data_path is None:
    print("❌ Dossier de données non trouvé!")
    print("📁 Contenu de RAW_DIR:")
    if RAW_DIR.exists():
        for item in RAW_DIR.iterdir():
            print(f"   - {item.name}")
    raise FileNotFoundError("Veuillez vérifier que les datasets sont téléchargés")

print(f"📂 Dossier de données: {data_path}")

# Statistiques des classes
classes = sorted([d.name for d in data_path.iterdir() if d.is_dir()])
print(f"\n📊 Nombre de classes: {len(classes)}")

# Compter les images par classe
class_counts = {}
for class_name in tqdm(classes, desc="Comptage"):
    class_dir = data_path / class_name
    count = len(list(class_dir.glob('*.jpg'))) + len(list(class_dir.glob('*.JPG'))) + len(list(class_dir.glob('*.png')))
    class_counts[class_name] = count

# DataFrame pour analyse
df_classes = pd.DataFrame([
    {"class": k, "count": v} for k, v in class_counts.items()
])
df_classes = df_classes.sort_values('count', ascending=False)

print(f"\n📈 Total d'images: {df_classes['count'].sum()}")
print(f"📈 Moyenne par classe: {df_classes['count'].mean():.0f}")
print(f"📈 Min: {df_classes['count'].min()}")
print(f"📈 Max: {df_classes['count'].max()}")

# Visualiser la distribution
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Distribution des classes
ax1 = axes[0]
df_classes.plot(x='class', y='count', kind='bar', ax=ax1, legend=False)
ax1.set_title('Distribution des images par classe', fontsize=14)
ax1.set_xlabel('Classe')
ax1.set_ylabel('Nombre d\'images')
ax1.tick_params(axis='x', rotation=90, labelsize=6)

# Histogramme
ax2 = axes[1]
ax2.hist(df_classes['count'], bins=30, edgecolor='black')
ax2.set_title('Histogramme des tailles de classes', fontsize=14)
ax2.set_xlabel('Nombre d\'images')
ax2.set_ylabel('Fréquence')
ax2.axvline(df_classes['count'].mean(), color='red', linestyle='--', label='Moyenne')
ax2.legend()

plt.tight_layout()
plt.savefig(PROJECT_DIR / 'class_distribution.png', dpi=150)
plt.show()

# Parser les noms de classes
def parse_class_name(name):
    """Extrait la plante et la condition du nom de classe"""
    parts = name.split('___')
    if len(parts) == 2:
        return {
            'plant': parts[0].replace('_', ' '),
            'condition': parts[1].replace('_', ' '),
            'is_healthy': 'healthy' in parts[1].lower()
        }
    return {
        'plant': name,
        'condition': 'unknown',
        'is_healthy': None
    }

# Analyser toutes les classes
class_info = {}
plants = set()
conditions = set()

for class_name in classes:
    info = parse_class_name(class_name)
    class_info[class_name] = info
    plants.add(info['plant'])
    conditions.add(info['condition'])

print(f"\n🌱 Plantes uniques: {len(plants)}")
for p in sorted(plants):
    print(f"  - {p}")

print(f"\n🏥 Conditions uniques: {len(conditions)}")

In [ ]:
# Visualiser des exemples
def show_samples(data_path, classes, samples_per_class=3):
    """Affiche des exemples de chaque classe"""
    n_classes = min(12, len(classes))
    fig, axes = plt.subplots(n_classes, samples_per_class, figsize=(samples_per_class * 3, n_classes * 2.5))
    
    for i, class_name in enumerate(classes[:n_classes]):
        class_dir = data_path / class_name
        images = list(class_dir.glob('*.jpg')) + list(class_dir.glob('*.JPG'))
        
        for j in range(samples_per_class):
            ax = axes[i, j] if n_classes > 1 else axes[j]
            
            if j < len(images):
                img = cv2.imread(str(images[j]))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                ax.imshow(img)
            
            ax.axis('off')
            if j == 0:
                ax.set_title(class_name[:30], fontsize=8)
    
    plt.tight_layout()
    plt.savefig(PROJECT_DIR / 'sample_images.png', dpi=150)
    plt.show()

show_samples(data_path, classes[:12])

# Vérifier la qualité des images
print("🔍 Vérification de la qualité des images...")

issues = []
image_sizes = []
image_formats = Counter()

for class_name in tqdm(classes[:10], desc="Vérification"):  # Limiter pour rapidité
    class_dir = data_path / class_name
    
    for img_path in list(class_dir.glob('*'))[:50]:  # 50 images par classe
        try:
            img = cv2.imread(str(img_path))
            if img is None:
                issues.append(f"Cannot read: {img_path}")
                continue
            
            h, w = img.shape[:2]
            image_sizes.append((w, h))
            image_formats[img_path.suffix.lower()] += 1
            
            # Vérifier les dimensions minimales
            if w < 50 or h < 50:
                issues.append(f"Too small ({w}x{h}): {img_path}")
            
        except Exception as e:
            issues.append(f"Error {e}: {img_path}")

print(f"\n📊 Formats trouvés: {dict(image_formats)}")
print(f"⚠️ Problèmes détectés: {len(issues)}")

if issues:
    print("\nExemples de problèmes:")
    for issue in issues[:5]:
        print(f"  - {issue}")

# Statistiques de taille
sizes = np.array(image_sizes)
print(f"\n📐 Tailles d'images:")
print(f"  Min: {sizes.min(axis=0)}")
print(f"  Max: {sizes.max(axis=0)}")
print(f"  Moyenne: {sizes.mean(axis=0).astype(int)}")

In [ ]:
# Préparer le dataset final
print("📦 Préparation du dataset final...")

# Structure cible
TRAIN_DIR = PROCESSED_DIR / 'train'
VAL_DIR = PROCESSED_DIR / 'val'
TEST_DIR = PROCESSED_DIR / 'test'

for d in [TRAIN_DIR, VAL_DIR, TEST_DIR]:
    if d.exists():
        shutil.rmtree(d)
    d.mkdir(parents=True)

# Ratios
TRAIN_RATIO = 0.7
VAL_RATIO = 0.15
TEST_RATIO = 0.15

# Copier les images
np.random.seed(42)

for class_name in tqdm(classes, desc="Copie des images"):
    src_dir = data_path / class_name
    images = list(src_dir.glob('*.jpg')) + list(src_dir.glob('*.JPG')) + list(src_dir.glob('*.png'))
    
    np.random.shuffle(images)
    
    n = len(images)
    n_train = int(n * TRAIN_RATIO)
    n_val = int(n * VAL_RATIO)
    
    splits = {
        'train': images[:n_train],
        'val': images[n_train:n_train + n_val],
        'test': images[n_train + n_val:]
    }
    
    for split_name, split_images in splits.items():
        dest_dir = PROCESSED_DIR / split_name / class_name
        dest_dir.mkdir(parents=True, exist_ok=True)
        
        for img_path in split_images:
            shutil.copy2(img_path, dest_dir / img_path.name)

# Vérifier le split
def count_split(split_dir):
    """Compte les images dans un split"""
    total = 0
    for class_dir in split_dir.iterdir():
        if class_dir.is_dir():
            total += len(list(class_dir.glob('*')))
    return total

print("\n📊 Résumé du split:")
print(f"  Train: {count_split(TRAIN_DIR)} images")
print(f"  Val: {count_split(VAL_DIR)} images")
print(f"  Test: {count_split(TEST_DIR)} images")

# Sauvegarder les métadonnées
metadata = {
    "created_at": pd.Timestamp.now().isoformat(),
    "source": "PlantVillage + New Plant Diseases",
    "num_classes": len(classes),
    "classes": classes,
    "class_info": class_info,
    "class_to_idx": {name: idx for idx, name in enumerate(classes)},
    "splits": {
        "train": count_split(TRAIN_DIR),
        "val": count_split(VAL_DIR),
        "test": count_split(TEST_DIR)
    },
    "plants": list(plants),
    "input_size": [224, 224],
    "data_path": str(data_path)
}

# Sauvegarder dans le dossier models
with open(MODELS_DIR / 'class_mapping.json', 'w', encoding='utf-8') as f:
    json.dump(metadata, f, indent=2, ensure_ascii=False)

print(f"✅ Métadonnées sauvegardées: {MODELS_DIR / 'class_mapping.json'}")

# Copier vers Google Drive si on est sur Colab
if IN_COLAB:
    import shutil
    drive_models = Path(DRIVE_BASE) / 'drone-agri-ai' / 'models'
    drive_models.mkdir(parents=True, exist_ok=True)
    shutil.copy(MODELS_DIR / 'class_mapping.json', drive_models)
    
    # Copier aussi les graphiques
    for img in ['class_distribution.png', 'sample_images.png']:
        src = PROJECT_DIR / img
        if src.exists():
            shutil.copy(src, drive_models.parent)
    
    print(f"✅ Fichiers copiés vers Google Drive: {drive_models.parent}")

print("\n" + "="*60)
print("✅ PRÉPARATION DES DONNÉES TERMINÉE")
print("="*60)
print(f"   Classes: {len(classes)}")
print(f"   Train: {count_split(TRAIN_DIR)} images")
print(f"   Val: {count_split(VAL_DIR)} images")
print(f"   Test: {count_split(TEST_DIR)} images")
print(f"\n➡️ Prochaine étape: Exécuter 02_model_training.ipynb")

## ✅ Données prêtes!
# 
### Le dataset est maintenant prêt pour l'entraînement:
### - Images organisées en train/val/test
### - Métadonnées sauvegardées
### - Copie sur Google Drive